---
comments: true
layout: post
title: Data Structures Writeup
description: .
type: tangibles
courses: { csp: {week: 26} }
---

## Collections
- Blog Python Model code and SQLite Database.

In [ ]:
## Python Model
class Design(db.Model):
    __tablename__ = 'designs'

    name = db.Column(db.String, unique=False, primary_key=True)
    type = db.Column(db.Text, unique=False, nullable=False)
    content = db.Column(db.String, unique=False)
    # Define a relationship in Notes Schema to userID who originates the note, many-to-one (many notes to one user)
    userID = db.Column(db.Integer, db.ForeignKey('users.id'))
    likes = db.Column(db.Integer, unique=False, nullable=False)
    dislikes = db.Column(db.Integer, unique=False, nullable=False)
    description = db.Column(db.String, unique=False, nullable=False)
    def __init__(self, id, type, content, name, likes=0, dislikes=0, description=""):
        self.userID = id
        self.type = type
        self.content = content
        self.name = name
        self.likes = likes
        self.dislikes = dislikes
        self.description = description

    # Returns a string representation of the Notes object, similar to java toString()
    # returns string
    def __repr__(self):
        return {"Name": self.name, "Content": self.content, "Type": self.type, "Owner": self.userID, "Likes": self.likes, "Dislikes": self.dislikes, "Description": self.description,}

    # CRUD create, adds a new record to the Notes table
    # returns the object added or None in case of an error
    def create(self):
        try:
            # creates a Notes object from Notes(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Notes table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read, returns dictionary representation of Notes object
    # returns dictionary
    def read(self):
        
        return {
            "Name": self.name,
            "Owner": self.userID,
            "Content": self.content,
            "Type": self.type,
            "Likes": self.likes,
            "Dislikes": self.dislikes,
            "Description": self.description,
        }
    
    def update(self, name="", content="", type="", likes=0, dislikes=0, description=''):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(content) > 0:
            self.content = content
        if len(type) > 0:
            self.type = type
        if likes != 0:
            self.likes += likes
        if dislikes != 0:
            self.dislikes += dislikes
        if len(description) != 0:
            self.description = description
        db.session.commit()
        return self
    
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None


- From VSCode using SQLite3 Editor, show your unique collection/table in database, display rows and columns in the table of the SQLite database.
    - ![Screenshot 1](../../../images/sqldata.png)
- From VSCode model, show your unique code that was created to initialize table and create test data.
    - See Code Below
    - Code initializes three users, two default ones as requested by teacher, and an admin account for personal use.

In [ ]:
# User Initialization Code
# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', type="admin")
        u2 = User(name='Nicholas Tesla', uid='niko', password='123niko', type="standard")
        u3 = User(name='Admin_User', uid='admin', type="admin")
        users = [u1, u2, u3]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add a few 1 to 4 notes per user'''
                '''add user/post data to table'''
                user.create()
            except IntegrityError:
                '''fails with bad or duplicate data'''
                db.session.remove()
                print(f"Records exist, duplicate email, or error: {user.uid}")

## Lists and Dictionaries
Blog Python API code and use of List and Dictionaries.

- In VSCode using Debugger, show a list as extracted from database as Python objects.
    - GET request is sent to backend to search for all public designs. Backend fetches all public designs into a list in python debugger called design_return (red line). List contains all designs as python objects (red line).
    ![Screenshot 2](../../../images/listdebug.png)
- In VSCode use Debugger and list, show two distinct example examples of dictionaries, show Keys/Values using debugger.
    - After design is fetched, it is stored in python as a dictionary. This is viewable from the debugger as shown below. The purple values on the left are the keys, the values are shown on the right.
    ![Screenshot 3](../../../images/dictionary1.png)
    - When a design is created, the frontend sends an object with data regarding the design, such as its name and description. The backend then stores this data as a dictionary with keys in purple on the left and values on the right.
    ![Screenshot 4](../../../images/dictionary2.png)

## APIs and JSON
Blog Python API code and use of Postman to request and respond with JSON.

- In VSCode, show Python API code definition for request and response using GET, POST, UPDATE methods. Discuss algorithmic condition used to direct request to appropriate Python method based on request method.
    - Within the code shown above, the API contains several CRUDs, such as a CRUD for modifying users and one for modifying Designs. 
    - A resource is then added to the API under the appropriate link.
    - When a request is sent to the link, the appropriate function is called according to the type of request send.

In [ ]:
    api.add_resource(_CRUD, '/')
    api.add_resource(_DesignCRUD, '/design')

- In VSCode, show algorithmic conditions used to validate data on a POST condition.
    - Algorithmic conditions ensure that inputted data is valid. The following two conditions are part of the user creation code. They ensure that the password is secure by ensuring that it is longer than a certain length, and ensure that a Name and password exists.

In [ ]:
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 400
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            

- In Postman, show URL request and Body requirements for GET, POST, and UPDATE methods.
- In Postman, show the JSON response data for 200 success conditions on GET, POST, and UPDATE methods.
![Screenshot 5](../../../images/post.png)
![Screenshot 6](../../../images/get.png)
![Screenshot 7](../../../images/put.png)
- In Postman, show the JSON response for error for 400 when missing body on a POST request.
![Screenshot 8](../../../images/posterror.png)
In Postman, show the JSON response for error for 404 when providing an unknown user ID to a UPDATE request.
- Updated user api code automatically retrieves user ID from JWT token by decoding it.
- This means that no UserID needs to be provided, simply a name and password
- This also ensures that users can only update their own user
- See line below

In [ ]:
cur_user = jwt.decode(token, current_app.config["SECRET_KEY"], algorithms=["HS256"])['_uid']